In [1]:
import pandas as pd 

customers = pd.read_csv('customers.csv')
orders = pd.read_csv('orders.csv')
order_details = pd.read_csv('order_details.csv')
product = pd.read_csv('product.csv')


### How many customers do we have in the data?

In [2]:


df_1 = _deepnote_execute_sql("""SELECT count(DISTINCT (customer_id))
FROM customers
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_1

,count(DISTINCT customer_id)
0,795


### What was the city with the most profit for the company in 2015 and how much was it?

In [3]:


df_2 = _deepnote_execute_sql("""SELECT o.shipping_city AS city
	,sum(od.order_profits) AS profit
FROM orders o
INNER JOIN order_details od using (order_id)
WHERE order_date LIKE '%2015'
GROUP BY 1
ORDER BY 2 DESC 
LIMIT 1
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_2

,city,profit
0,New York City,14753


### How many different cities do we have in the data?

In [4]:


df_3 = _deepnote_execute_sql("""SELECT count(DISTINCT (shipping_city))
FROM orders
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_3

,count(DISTINCT shipping_city)
0,531


### Show the total spent by customers from low to high

In [5]:


df_4 = _deepnote_execute_sql("""SELECT o.customer_id
	,sum(od.order_sales)
FROM orders o
INNER JOIN order_details od using (order_id)
GROUP BY 1
ORDER BY 2 ASC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_4

,customer_id,sum(od.order_sales)
0,456,5
1,738,5
2,546,16
3,124,17
4,657,22
...,...,...
788,758,14596
789,623,15117
790,388,16744
791,731,19050


### What is the most profitable city in the state of Tennessee?

In [6]:


df_5 = _deepnote_execute_sql("""SELECT city
	,max(total_profits)
FROM (
	SELECT o.shipping_city AS city
		,sum(od.order_profits) AS total_profits
	FROM orders o
	INNER JOIN order_details od ON o.order_id = od.order_id
	WHERE o.shipping_state = 'Tennessee'
	GROUP BY 1
	)
    GROUP BY 1
    ORDER BY 2 DESC

""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_5

,city,max(total_profits)
0,Lebanon,83
1,Chattanooga,28
2,Smyrna,20
3,Murfreesboro,15
4,Johnson City,12
5,Bartlett,10
6,Jackson,-44
7,Hendersonville,-46
8,Franklin,-93
9,Bristol,-127


### What's the average annual profit for that city (Lebanon) across all years in that city?

In [7]:


df_6 = _deepnote_execute_sql("""SELECT avg(od.order_profits)
FROM order_details od
INNER JOIN orders o using (order_id)
WHERE o.shipping_city = 'Lebanon'
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_6

,avg(od.order_profits)
0,27.666667


### What is the distribution of customer types in the data?

In [8]:


df_7 = _deepnote_execute_sql("""select count(*) from customers
where customer_segment = 'Corporate'
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_7

,count_star()
0,237


### What's the most profitable product category on average in Iowa across all years?

In [9]:


df_8 = _deepnote_execute_sql("""SELECT p.product_category AS category
	,avg(od.order_profits) AS profit
FROM product p
INNER JOIN order_details od ON p.product_id = od.product_id
INNER JOIN orders o ON o.order_id = od.order_id
WHERE o.shipping_state = 'Iowa'
GROUP BY 1
ORDER BY 2 DESC

""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_8

,category,profit
0,Furniture,130.250000
1,Technology,79.750000
2,Office Supplies,15.727273


### What is the most popular product in that category across all states in 2016?

In [10]:


df_10 = _deepnote_execute_sql("""SELECT p.product_name
	,sum(od.order_sales)
FROM product p
INNER JOIN order_details od ON p.product_id = od.product_id
INNER JOIN orders o ON od.order_id = o.order_id
WHERE right(o.order_date, 4) LIKE '%2016'
	AND p.product_category = 'Furniture'
GROUP BY 1
ORDER BY 2 DESC

""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_10

,product_name,sum(od.order_sales)
0,"Riverside Palais Royal Lawyers Bookcase, Royal...",11717
1,HON 5400 Series Task Chairs for Big and Tall,5257
2,Bush Advantage Collection Racetrack Conference...,4412
3,Chromcraft Bull-Nose Wood Oval Conference Tabl...,4298
4,Hon Olson Stacker Chairs,3945
...,...,...
265,Eldon Pizzaz Desk Accessories,9
266,Eldon Image Series Black Desk Accessories,8
267,"Eldon Regeneration Recycled Desk Accessories, ...",8
268,"DAX Charcoal/Nickel-Tone Document Frame, 5 x 7",8


### Which customer got the most discount in the data? (in total amount)

In [11]:


df_9 = _deepnote_execute_sql("""SELECT c.customer_id
	,c.customer_name
	,sum(od.order_sales / (1 - od.order_discount) - od.order_sales) AS total_discount
FROM customers c
INNER JOIN orders o using (customer_id)
INNER JOIN order_details od ON o.order_id = od.order_id
GROUP BY 1
	,2
ORDER BY 3 DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_9

,customer_id,customer_name,total_discount
0,687,Sean Miller,23929.083333
1,166,Cindy Stewart,11594.333333
2,478,Luke Foster,9052.166667
3,308,Grant Thornton,8746.833333
4,330,Henry Goldwyn,7692.583333
...,...,...,...
788,627,Ricardo Sperren,0.000000
789,649,Roland Murray,0.000000
790,292,Fred Wasserman,0.000000
791,742,Theresa Coyne,0.000000


### How widely did monthly profits vary in 2018?

In [12]:


df_11 = _deepnote_execute_sql("""with mp as (
    select cast(substr(o.order_date,1, Instr(o.order_date, '/')-1) as int) as months,
sum(od.order_profits) as monthly_profit
from orders o
join order_details od using (order_id)
where right(o.order_date, 4) like '%2018'
group by 1 
order by 1 asc 
)

select *,
lead(monthly_profit) over () as next_month,
monthly_profit - lead(monthly_profit) over () as diff
from mp 
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_11

,months,monthly_profit,next_month,diff
0,1,7137,1612.0,5525.0
1,2,1612,14758.0,-13146.0
2,3,14758,934.0,13824.0
3,4,934,6342.0,-5408.0
4,5,6342,8226.0,-1884.0
5,6,8226,6951.0,1275.0
6,7,6951,9034.0,-2083.0
7,8,9034,10987.0,-1953.0
8,9,10987,9272.0,1715.0
9,10,9272,9217.0,55.0


### Which order was the highest in 2015?

In [13]:


df_12 = _deepnote_execute_sql("""SELECT od.order_id
	,od.order_sales
FROM order_details od
INNER JOIN orders o using (order_id)
WHERE right(o.order_date, 4) LIKE '%2015'
ORDER BY 2 DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_12

,order_id,order_sales
0,CA-2015-145317,22638
1,CA-2015-116904,9450
2,CA-2015-143917,8188
3,CA-2015-139892,8160
4,CA-2015-145541,7000
...,...,...
1998,CA-2015-154158,1
1999,CA-2015-159310,1
2000,CA-2015-165309,1
2001,CA-2015-169257,1


### What was the rank of each city in the East region in 2015?

In [14]:


df_13 = _deepnote_execute_sql("""

select o.shipping_region, o.shipping_city as city, sum(od.quantity) as qty
from orders o 
join order_details od using (order_id)
where o.shipping_region = 'East' and 
right(o.order_date, 4) like '%2015'
group by 1,2 
order by 3 DESC


""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_13

,shipping_region,city,qty
0,East,New York City,1708
1,East,Philadelphia,403
2,East,Columbus,167
3,East,Newark,64
4,East,Fairfield,53
5,East,Long Beach,44
6,East,Lakewood,38
7,East,Lancaster,36
8,East,Lawrence,31
9,East,Dover,30


In [15]:


df_15 = _deepnote_execute_sql("""
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_15

### Join all tables in the database together

In [17]:


df_14 = _deepnote_execute_sql("""SELECT c.*
	,o.*
	,od.*
	,p.*
FROM customers c
FULL JOIN orders o ON c.customer_id = o.customer_id
FULL JOIN order_details od ON o.order_id = od.order_id
FULL JOIN product p ON od.product_id = p.product_id

""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_14

,customer_id,customer_name,customer_segment,order_id,customer_id,order_date,shipping_city,shipping_state,shipping_region,shipping_country,...,quantity,order_discount,order_profits,order_profit_ratio,order_sales,product_id,product_name,product_category,product_subcategory,product_manufacturer
0,1,Aaron Bergman,Consumer,CA-2017-140935,1.0,11/10/2017,Oklahoma City,Oklahoma,Central,United States,...,2.0,0.0,55.0,0.16,342.0,1432.0,"Sauder Facets Collection Library, Sky Alder Fi...",Furniture,Bookcases,Sauder
1,2,Aaron Hawkins,Corporate,US-2015-158400,2.0,10/25/2015,San Francisco,California,West,United States,...,4.0,0.2,19.0,0.38,49.0,716.0,GBC VeloBind Cover Sets,Office Supplies,Binders,GBC
2,3,Aaron Smayling,Corporate,US-2018-147655,3.0,9/4/2018,Redmond,Oregon,West,United States,...,7.0,0.7,-59.0,-0.67,88.0,270.0,"Avery Trapezoid Extra Heavy Duty 4"" Binders",Office Supplies,Binders,Avery
3,4,Adam Bellavance,Home Office,US-2017-108637,4.0,3/13/2017,Waynesboro,Virginia,South,United States,...,2.0,0.0,45.0,0.28,160.0,612.0,"Eureka The Boss Lite 10-Amp Upright Vacuum, Blue",Office Supplies,Appliances,Eureka
4,5,Adam Hart,Corporate,CA-2018-165029,5.0,11/26/2018,Atlanta,Georgia,South,United States,...,3.0,0.0,4.0,0.29,13.0,1164.0,Newell 347,Office Supplies,Art,Newell
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,260,Emily Phan,Consumer,CA-2016-163587,260.0,3/14/2016,Dover,New Hampshire,East,United States,...,4.0,0.0,8.0,0.46,17.0,183.0,Avery 476,Office Supplies,Labels,Avery
9995,260,Emily Phan,Consumer,CA-2016-123330,260.0,6/21/2016,Deltona,Florida,South,United States,...,3.0,0.2,38.0,0.35,108.0,927.0,Innergie mMini Combo Duo USB Travel Charging Kit,Technology,Phones,Other
9996,260,Emily Phan,Consumer,CA-2015-134278,260.0,7/6/2015,New York City,New York,East,United States,...,1.0,0.2,175.0,0.31,560.0,400.0,Canon Imageclass D680 Copier / Fax,Technology,Copiers,Canon
9997,794,Zack Gibbens,Consumer,None,NaN,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5e56f78f-de55-4d8b-8d05-1371d91890cf' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>